In [1]:
import pickle
from datetime import datetime
import json

In [2]:
folder = 'data/ple'

In [3]:
data_inicio = datetime.strptime('24/08/2020 00:00:00', '%d/%m/%Y %H:%M:%S')
turmas_validas = [12, 13, 14, 16, 17, 18, 20, 22, 23]

# Remove professores e usuarios que já tenham utilizado o sistema em outra turma
professores = Professor.objects.all().values_list('user')
usuarios_tentativa_anterior = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas,
                                                     timestamp__lt=data_inicio).values_list('user').distinct()

/home/lmoraes/.pyenv/versions/machineteaching/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1420: RuntimeWarning: DateTimeField UserLog.timestamp received a naive datetime (2020-08-24 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"


In [4]:
# O que fazer com alunos repetentes?
# Vou separá-los por enquanto

In [5]:
%%time
userlog = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class')
userlog.count()

CPU times: user 40.6 ms, sys: 11 ms, total: 51.6 ms
Wall time: 922 ms


46219

In [7]:
with open("%s/userlogs.pkl" % folder, "wb") as pklfile:
    pickle.dump(list(userlog), pklfile)

In [10]:
%%time
userlog_complete = UserLog.objects.filter(user__userprofile__user_class__in=turmas_validas
                                ).exclude(user__in=usuarios_tentativa_anterior
                                ).exclude(user__in=professores).order_by('timestamp'
                                ).values('user__id', 'problem__id', 'outcome', 
                                         'timestamp', 'user__userprofile__user_class',
                                         'solution', 'seconds_in_page', 'seconds_in_code', 
                                         'seconds_to_begin')
userlog_complete.count()

with open("%s/userlogs_complete.pkl" % folder, "wb") as pklfile:
    pickle.dump(list(userlog), pklfile)

CPU times: user 116 ms, sys: 56.6 ms, total: 173 ms
Wall time: 330 ms


In [7]:
%%time
problems = set(list(userlog.values_list("problem__id", flat=True).order_by("id").distinct()))

CPU times: user 58.1 ms, sys: 980 µs, total: 59.1 ms
Wall time: 1.5 s


In [8]:
# One row per test case
solutions = Solution.objects.filter(problem__in=problems, ignore=False).values(
    "problem_id", "content", "header", "problem__testcase__content").order_by("content")

In [10]:
%%time
data = []
old_content = ""
for item in solutions:
    # If it is the same problem, keep adding test cases
    if item["content"] == old_content:
        data[-1]["test_case"].append(json.loads(item["problem__testcase__content"]))
    # Otherwise, start new row
    else:
        item_data = {
            "problem_id": item["problem_id"],
            "solution": item["content"],
            "function_call": item["header"],
            "test_case": [json.loads(item["problem__testcase__content"])]
        }
        data.append(item_data)
    old_content = item["content"]

CPU times: user 18.7 ms, sys: 0 ns, total: 18.7 ms
Wall time: 1.37 s


In [12]:
%%time
# Transform it into dict to lookup for problem
data_dict = {}
for item in data:
    data_dict[item["problem_id"]] = item

CPU times: user 15 µs, sys: 11 µs, total: 26 µs
Wall time: 28.6 µs


In [ ]:
# Calculate test case percentage for each userlog
userlog_testcase = []
for item in userlog:
    tcs = data[userlog.problem_id]["test_case"]
    correct = 0
    # Calculate student response and solution for each test case for each log
    for tc in tcs:
        # Transform solution into Python function
        function_obj = compile(data[userlog.problem_id]["solution"], 'solution', 'exec')
        exec(function_obj)
        answer_solution = eval(item["function_call"])(*tc)
        
        # Transform student solution into Python function
        
        function_obj = compile(userlog.solution, 'solution', 'exec')
        exec(function_obj)
        answer_student = eval(item["function_call"])(*tc)

In [ ]:
for item in data:
    for tc in item["test_case"]:
        print(tc)
        # Transform solution into python function
        function_obj = compile(item["solution"], 'solution', 'exec')
        exec(function_obj)
        answer = eval(item["function_call"])(*tc)
        